In [7]:
import numpy as np
import pandas as pd
import random
import string
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import mean_squared_error


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from tqdm import tqdm

In [2]:
# import numpy as np
# import pandas as pd
# from sklearn.datasets import make_classification

# # Set random seed for reproducibility
# np.random.seed(42)

# # Define the number of groups and the number of observations per group
# num_groups = 5
# observations_per_group = 100

# # Generate random group labels
# group_labels = np.repeat(np.arange(num_groups), observations_per_group)

# # Generate random features
# X, _ = make_classification(n_samples=observations_per_group * num_groups, n_features=10, random_state=42)

# # Generate random fixed effects for each group
# group_effects = np.random.randn(num_groups)

# # Generate random random effects for each observation within each group
# random_effects = np.random.randn(observations_per_group * num_groups)

# # Compute the target variable with mixed effects
# y = np.dot(X, np.random.randn(X.shape[1])) + group_effects[group_labels] + random_effects

# # Create a pandas DataFrame to store the data
# data = pd.DataFrame(X)
# data['Group'] = group_labels
# data['Fixed_Effect'] = group_effects[group_labels]
# data['Random_Effect'] = random_effects
# data['Target'] = y

# # Print the first few rows of the dataset
# print(data.head())


          0         1         2         3         4         5         6  \
0 -0.831822  3.778560 -0.916192 -0.690541 -0.715760 -0.825560 -4.887584   
1 -0.066133 -0.440522 -0.436720  0.265878 -0.247026  0.140124  1.030778   
2 -0.437458 -1.526773 -0.887492  0.419532 -0.372833  0.013308 -1.389822   
3 -0.526035 -0.811048  0.492019  0.420973  1.097153  0.174118  1.016694   
4 -0.305225 -0.054340 -1.065114 -1.311836 -0.186971 -0.011333 -0.173507   

          7         8         9  Group  Fixed_Effect  Random_Effect    Target  
0 -0.783766 -2.016605 -0.067178      0      0.496714      -0.234137  1.860707  
1 -0.470042  0.527629  2.099722      0      0.496714       1.579213  1.373105  
2 -0.120948 -1.320396  0.722381      0      0.496714       0.767435  4.362660  
3  0.698828  0.412293 -2.153343      0      0.496714      -0.469474  0.144428  
4  0.010353 -0.125711 -0.609512      0      0.496714       0.542560  2.700925  


In [92]:
import numpy as np
import pandas as pd

# Set the random seed for reproducibility
np.random.seed(np.random.randint(0,200, size=1))

# Define the number of schools and students
num_schools = 20
students_per_school = 1000

# Generate random school IDs for each student
school_ids = np.repeat(np.arange(num_schools), students_per_school)

# Generate random student IDs within each school
student_ids = np.tile(np.arange(students_per_school), num_schools)

# Generate random student scores
student_scores = np.random.normal(0, 1, size=num_schools * students_per_school)

# Generate random student ages
student_ages = np.random.randint(15, 20, size=num_schools * students_per_school)

# Generate nested ID columns
num_nested_ids = 50  # Number of nested ID columns
nested_ids = []
for i in range(num_nested_ids):
    nested_ids.append(np.random.randint(1, 10, size=num_schools * students_per_school))

# Create a DataFrame to store the dataset
data = pd.DataFrame({'SchoolID': school_ids, 'StudentID': student_ids, 'Score': student_scores,
                     'Age': student_ages})

# Add nested ID columns to the DataFrame
for i in range(num_nested_ids):
    column_name = f'NestedID{i+1}'
    data[column_name] = nested_ids[i]

# Print the first few rows of the dataset
data.head()

,SchoolID,StudentID,Score,Age,NestedID1,NestedID2,NestedID3,NestedID4,NestedID5,NestedID6,...,NestedID41,NestedID42,NestedID43,NestedID44,NestedID45,NestedID46,NestedID47,NestedID48,NestedID49,NestedID50
0,0,0,-0.091950,17,9,2,1,2,2,6,...,4,3,2,5,8,8,3,3,6,2
1,0,1,-1.463351,18,1,4,2,3,1,6,...,7,8,7,7,9,9,8,5,8,9
2,0,2,1.081792,16,4,9,2,8,9,3,...,8,4,1,6,9,5,5,9,7,3
3,0,3,-0.239325,18,8,8,8,2,2,8,...,7,7,1,3,2,6,1,5,9,6
4,0,4,-0.491129,15,6,6,3,3,8,2,...,6,9,5,6,7,3,5,8,9,2


In [83]:
data.shape

(20000, 54)

In [84]:
grouping_col = list(f"NestedID{i}" for i in range(1,51))
grouping_col.append("SchoolID")
grouping_col.append("StudentID")

In [85]:
from itertools import combinations
combi = list(combinations(grouping_col, 4))
len(combi)

270725

In [96]:
accuracyList = []
for i, selCol in tqdm(enumerate(combi)):
    
    X = pd.concat([data[list(selCol)], data.loc[:,"Age"]], axis=1)
    y = data['Score']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    pipeline = make_pipeline(MinMaxScaler(), LinearRegression())
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)
    accuracyList.append(mean_squared_error(y_test, y_pred))

270725it [48:42, 92.64it/s] 


In [97]:
idx = accuracyList.index(min(accuracyList))
combi[idx], min(accuracyList)

(('NestedID1', 'NestedID5', 'NestedID27', 'NestedID42'), 1.0301894282343966)

In [93]:
def regression_check(dataset, keep_nested=True):
    
    if keep_nested==True:
        X = data.drop("Score", axis=1)
        y = data["Score"]
    else:
        X = data[["StudentID", "SchoolID", "Age"]]
        y = data["Score"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    pipeline = make_pipeline(MinMaxScaler(), LinearRegression())
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    return y_pred, y_test, mse

In [100]:
pred, test, mse = regression_check(data, keep_nested=False)

In [101]:
mse

1.0315282539114623

In [103]:
pred

array([ 0.01378634, -0.00959229,  0.00753968, ...,  0.00574987,
        0.01501943,  0.00268776])

In [104]:
test

10650    2.122163
2041    -0.052361
8668     0.464311
1114    -0.264617
13902    0.934375
           ...   
5388     0.266773
749     -1.130119
611      0.969695
13637   -0.577886
11617   -0.412139
Name: Score, Length: 6600, dtype: float64